In [ ]:
    '''
    if (repo_path is not None):
        print("Must specify repository path.");
        return False;
    '''

In [82]:
from datetime import datetime;
import os;
import re;
import subprocess;
import sys;

In [83]:
#repo_path = "/home/cam/P1/";
#repo_path = "/mnt/hgfs/cam/Documents/tamu/research/project_evaluation/P4-master/P4/";
repo_path = "/mnt/hgfs/cam/Documents/tamu/course/csce-670/term_proj/SocialRec/Git/SocialRec/";
#repo_path = "/mnt/hgfs/cam/Documents/tamu/course/csce-670/term_proj/SocialRec/Git/BAD";
#repo_path = sys.argv[1];

In [84]:
def isGitRepo(repo_path):
    
    if (not os.path.exists(repo_path + "/.git")):
        print("'" + repo_path + "'" + " is not a git repository");
        return False;
    
    return True;

In [85]:
class Student:
    
    students_list = set();
    
    def __init__(self, name, email):
        
        self.name = name;
        self.email = email;
        self.submissions = set();
        self.addToStudentsList();
    
    def addToStudentsList(self):
        
        if (self not in Student.students_list):
            Student.students_list.add(self);
        else:
            return;
    
    def addSubmission(self, submission):
        
        if (submission not in self.submissions):
            self.submissions.add(submission);
        else:
            return;

In [86]:
class Commit:
    
    def __init__(self, log_format, log_stat, files):
        
        self.commit_hash = log_format["commit_hash"];
        self.author_name = log_format["author_name"];
        self.author_email = log_format["author_email"];
        self.author_date = log_format["author_date"];
        self.committer_name = log_format["committer_name"];
        self.subject = log_format["subject"];
        #(self.num_insertions, self.num_deletions, self.files_changed) = self.getFileStats(log_stat);
        (self.num_insertions, self.num_deletions, self.num_files_changed) = self.getFileStats(log_stat);
        self.files = files;
        self.num_lines_changed = self.num_insertions + self.num_deletions;
        #self.num_files_changed = len(self.files_changed);
    
    def getFileStats(self, log_stat):
        
        files_changed_regex = r"[0-9][0-9]*\sfiles*\schanged";
        insertions_regex = r"[0-9][0-9]*\sinsertions*\(\+\)";
        deletions_regex = r"[0-9][0-9]*\sdeletions*\(\-\)";
        
        files_changed_str = str(re.findall(files_changed_regex, log_stat));
        insertions_str = str(re.findall(insertions_regex, log_stat));
        deletions_str = str(re.findall(deletions_regex, log_stat));
        
        num_files_changed = self.extractDigit(files_changed_str);                                
        num_insertions = self.extractDigit(insertions_str);
        num_deletions = self.extractDigit(deletions_str);
        
        #files_changed = self.getUpdatedFilesList(log_stat);
        
        return (num_insertions, num_deletions, num_files_changed);
    
    def extractDigit(self, in_string):
        
        digit = re.findall((r"[0-9][0-9]*"), in_string);
        
        if (len(digit) > 0):
            return int(digit[0]);
        else:
            return 0;
    
    def getUpdatedFilesList(self, log_stat):
        
        start_is = [start_i.start() for start_i in re.finditer(r"\n\s.*\s\|\s+[a-zA-Z0-9]", log_stat)];
        end_is = [end_i.start() for end_i in re.finditer(r"\s\|\s+[a-zA-Z0-9]", log_stat)];
        
        file_list = list();
        
        max_i = (len(start_is) + len(end_is)) / 2;
        for i in range(0,max_i):
            
            start_i = start_is[i] + len("\n ");
            end_i = end_is[i];
            file_name = log_stat[start_i:end_i];
            file_name = file_name.strip();
            file_list.append(file_name);
        
        return file_list;

In [87]:
class Submission():
    
    def __init__(self, repo_name, students, commits):
        
        self.repo_name = repo_name; # Name of this repository ("repo")
        self.students = students; # Name of student (as repo author)
        self.commits = commits; # List of commit objects for this repo
        self.tot_num_commits = len(commits); # Number of commit objects for this repo

In [88]:
COMMIT_FIELDS = ["commit_hash",
                 "author_name", "author_email", "author_date",
                 "committer_name", "committer_email", "committer_date",
                 "subject", "body", "notes"];

LOG_FORMAT = ["%H",
              "%an", "%ae", "%at",
              "%cn", "%ce", "%cd",
              "%s", "%b", "%N"];

In [89]:
def getRepoName(repo_path):
    
    repo_name = os.path.basename(os.path.normpath(repo_path));
    
    return repo_name;

In [90]:
def prepareLogFormatsOutput(log_format):
    
    log_format = log_format.strip("\n\x1e").split("\x1e");
    
    log_format = [line.strip().split("\x1f") for line in log_format];
    
    commits_formats = [dict(zip(COMMIT_FIELDS, line)) for line in log_format];
    
    return commits_formats;

In [91]:
def prepareLogStatsOutput(log_stat):
    
    start_is = [start_i.start() for start_i in re.finditer(r"commit\s\b[0-9a-f]{40}\b", log_stat)];
    
    commits_stats = list();
    
    max_i = len(start_is);
    for i in range(0, max_i):
        
        if (i < (max_i - 1)):
            start_i = i;
            end_i = i + 1;
            commit_stat = log_stat[start_is[start_i]:start_is[(end_i)]];
            commits_stats.append(commit_stat);
        elif (i == (max_i - 1)):
            start_i = i;
            end = len(log_stat) - 1;
            commit_stat = log_stat[start_is[start_i]:end];
            commits_stats.append(commit_stat);
    
    commits_stats_dict = dict();
    
    for commit_stat in commits_stats:
        
        commit_hash = re.findall(r"\b[0-9a-f]{40}\b", commit_stat)[0];
        commits_stats_dict[commit_hash] = commit_stat;
    
    return commits_stats_dict;

In [92]:
def parseGitLog(repo_path):
    
    log_format = "%x1f".join(LOG_FORMAT) + "%x1e";
    
    gd = "--git-dir=" + str(repo_path) + ".git/";
    wt = "--work-tree=" + str(repo_path);
    f = "--format=" + str(log_format);
    stat = "--stat";
    
    proc = subprocess.Popen(("git %s %s log %s" % (gd,wt,f)), stdout=subprocess.PIPE, shell=True);
    (log_format, _) = proc.communicate();
    log_formats = prepareLogFormatsOutput(log_format);
    
    proc = subprocess.Popen(("git %s %s log %s" % (gd,wt,stat)), stdout=subprocess.PIPE, shell=True);
    (log_stat, _) = proc.communicate();
    log_stats = prepareLogStatsOutput(log_stat);


    return log_formats, log_stats;

In [93]:
def prepareDiffTreeOutput(diff_tree):
    
    changed_files = list();
    
    for file_name in diff_tree.splitlines():
        changed_files.append(str(file_name));
    
    return changed_files;

In [94]:
def parseGitDiffTree(commit_hash, diff_filter):
    
    gd = "--git-dir=" + str(repo_path) + ".git/";
    wt = "--work-tree=" + str(repo_path);    
    nci = "--no-commit-id";
    no = "--name-only";
    df = "--diff-filter=" + str(diff_filter);
    r = "-r";
    ch = str(commit_hash);
    
    proc = subprocess.Popen(("git %s %s diff-tree %s %s %s %s %s" % (gd,wt,nci,no,df,r,ch)), stdout=subprocess.PIPE, shell=True);
    (diff_tree, _) = proc.communicate();
    diff_filter_files = prepareDiffTreeOutput(diff_tree);
    
    return diff_filter_files;

In [95]:
def getCommitFiles(commit_hash):
    
    diff_filters = {"added":"A",
                    "copied":"C",
                    "deleted":"D",
                    "modified":"M",
                    "renamed":"R"};
    
    files = dict();
    
    for diff_filter in diff_filters:
        
        df = diff_filters.get(diff_filter);
        diff_filter_files = parseGitDiffTree(commit_hash, df);
        files[diff_filter] = diff_filter_files;
        
    return files;

In [96]:
def getStudents(repo_commits):
    
    students = set();
    existing = set();
    
    for commit in repo_commits:
        
        name = commit.author_name;
        email = commit.author_email;
        
        name_email = (name, email);
        
        if (name_email not in existing):
            student = Student(name, email);
            students.add(student);
            existing.add(name_email);
    
    return students;

In [97]:
def getGitCommits():
    
    commits = list();
    
    (log_formats, log_stats) = parseGitLog(repo_path);
    
    for log_format in log_formats:
        
        commit_hash = log_format["commit_hash"];
        log_stat = log_stats[commit_hash];
        updated_files = getCommitFiles(commit_hash);
        
        commit = Commit(log_format, log_stat, updated_files);
        if (commit not in commits):
            commits.append(commit);
    
    return commits;

In [159]:
def tempName(repo_path):
    
    commits = getGitCommits();
    
    repo_name = getRepoName(repo_path);
    
    students = getStudents(commits);
    
    submission = Submission(repo_name, students, commits);
    
    r = getCommitsRange(submission, "Jun 1 2017  1:33:12PM");#, "Jun 1 2005  1:33:12PM");
    print(r);
    
    '''
    import io, json;
    with io.open('data.txt', 'w', encoding='utf-8') as f:
        f.write(json.dumps(commits, ensure_ascii=False));
    '''
    
    for student in students:
        student.addSubmission(submission);
    
    printInfo(commits, students);

In [160]:
def printInfo(commits, students):
    
    print("::ALL COMMITS::\n\n");
    for commit in commits:
        attrs = vars(commit);
        print('\n'.join("%s: %s" % item for item in attrs.items()));
        print("\n\n");
        
    print("::ALL STUDENTS::\n\n");
    for student in students:
        attrs = vars(student);
        print('\n'.join("%s: %s" % item for item in attrs.items()));
        print("\n\n");
        
    print("::ALL SUBMISSIONS::\n\n");
    for student in students:
        submissions = student.submissions;
        for submission in submissions:
            attrs = vars(submission);
            print('\n'.join("%s: %s" % item for item in attrs.items()));
            print("\n\n");

In [161]:
def generateAnalysis():
    
    for assignment in assignments:
        assignmentStats = generateAssignmentStats(assignment);
        # do something with the stats

In [162]:
def generateAssignmentStats(assignment):
    # do something
    return assignment_stats;

In [163]:
def main():
    
    if (not isGitRepo(repo_path)):
        sys.exit();
        return;
    
    tempName(repo_path);

In [164]:
def getCommitsRange(submission, start_date=None, end_date=None):
    
    #datetime_object = datetime.strptime('Jun 1 2005  1:33PM', '%b %d %Y %I:%M%p')
    #('%Y-%m-%d %H:%M:%S');
    
    if (start_date is None and end_date is None):
        return submission.commits;
    
    if (start_date is None):
        start = 0;
        print("START is None!");
    else:
        start = getDateTime(start_date);
        print("START is: ",start);

    if (end_date is None):
        end = float("inf");
        print("END is None!");
    else:
        end = getDateTime(end_date);
        print("END is: ",end)
    
    commits_in_range = set();
    
    for commit in submission.commits:
        
        author_date = int(commit.author_date);
        
        if (author_date >= start and author_date <= end):
            commits_in_range.add(commit);
        #commit_datetime = datetime.fromtimestamp(int(dt)).strftime('%Y-%m-%d %H:%M:%S');
        #print(datetime_object);
        #print(commit.author_date);
        
    return commits_in_range;

In [165]:
def getDateTime(input_string):
    # 'Jun 1 2005  1:33:12PM'
    be = datetime.strptime(input_string, '%b %d %Y %I:%M:%S%p');#.utcfromtimestamp
    print(be);
    me = (be - datetime(1970,1,1)).total_seconds();
    #me = datetime.utcfromtimestamp(be);
    print(me);
    return me;

In [166]:
main();
#r = getCommitsRange(42342);
#print(r);

2017-06-01 13:33:12
1496323992.0
('START is: ', 1496323992.0)
END is None!
set([])
::ALL COMMITS::


files: {'deleted': [], 'renamed': [], 'copied': [], 'added': [], 'modified': []}
author_email: brtaylor1001@gmail.com
num_deletions: 0
num_insertions: 0
author_name: Brennen Taylor
num_lines_changed: 0
commit_hash: d071133c8105942a1d7b830e9c3c0d1797f764e1
num_files_changed: 0
author_date: 1493869368
committer_name: Brennen Taylor
subject: Merge branch 'master' of https://github.com/BrennenTaylor/SocialRec



files: {'deleted': ['biclustering_pruned_matrix.ipynb'], 'renamed': [], 'copied': [], 'added': [], 'modified': []}
author_email: brtaylor1001@gmail.com
num_deletions: 1406
num_insertions: 0
author_name: Brennen Taylor
num_lines_changed: 1406
commit_hash: 0f3afd640249c768484d178c25cfc1445733bc3d
num_files_changed: 1
author_date: 1493869362
committer_name: Brennen Taylor
subject: Remove last unused notebook file



files: {'deleted': [], 'renamed': [], 'copied': [], 'added': [], 'modi